In [1]:
import requests
import re 
import pandas as pd

In [2]:
def get_all_drugs(url, all_drugs=[]):
    """Get list of drugs from OpenFDA API.  
    
    Given a query URL, this function recursively calls the OpenFDA API and returns a list of dictionaries representing 
    all drugs returned by the query.
    """
    r = requests.get(url)
    if r.status_code == 200:
        n_drugs = r.json()['meta']['results']['total']
        drugs = r.json()['results']
        all_drugs.extend(drugs)
        progress = round(100 * (len(all_drugs) / n_drugs))
        print("{0}% of drugs fetched".format(progress))
        if 'Link' in r.headers:
            url = re.findall(r'<(.*)>', r.headers['Link'])[0] # extract the next page's url from response header
            return get_all_drugs(url, all_drugs) # recursively pass it through the function
        else: # the last page of results does not include a Link in the header 
            print("100% of drugs fetched")
            return all_drugs

Use the above function to get the raw drug data, flatten it to a dataframe, and write it to a file. This takes a few minutes to run. 

In [3]:
url = 'https://api.fda.gov/drug/label.json?search=_exists_:openfda.route&limit=1000'
drugs = get_all_drugs(url) # get list of drugs 

1% of drugs fetched
2% of drugs fetched
3% of drugs fetched
4% of drugs fetched
6% of drugs fetched
7% of drugs fetched
8% of drugs fetched
9% of drugs fetched
10% of drugs fetched
11% of drugs fetched
12% of drugs fetched
13% of drugs fetched
15% of drugs fetched
16% of drugs fetched
17% of drugs fetched
18% of drugs fetched
19% of drugs fetched
20% of drugs fetched
21% of drugs fetched
22% of drugs fetched
24% of drugs fetched
25% of drugs fetched
26% of drugs fetched
27% of drugs fetched
28% of drugs fetched
29% of drugs fetched
30% of drugs fetched
31% of drugs fetched
33% of drugs fetched
34% of drugs fetched
35% of drugs fetched
36% of drugs fetched
37% of drugs fetched
38% of drugs fetched
39% of drugs fetched
40% of drugs fetched
42% of drugs fetched
43% of drugs fetched
44% of drugs fetched
45% of drugs fetched
46% of drugs fetched
47% of drugs fetched
48% of drugs fetched
49% of drugs fetched
51% of drugs fetched
52% of drugs fetched
53% of drugs fetched
54% of drugs fetched


In [4]:
df = pd.json_normalize(drugs) # store them in a pandas dataframe
df.to_pickle('../data/raw/drugs.pkl') # save the dataframe as a pickle file to the data/raw directory